# 1D vertical rocket: symbolic model

This notebook introduces the symbolic 1D vertical rocket model
implemented in the `symrock` package.

We will:

1. Import the symbolic variables and equations of motion.
2. Inspect the structure of the ODE system.
3. Derive a simple mass evolution `m(t)` from `dm/dt = -mdot`.
4. Evaluate the Tsiolkovsky Δv formula for a simple example.


In [ ]:
import sympy as sp

from symrock import (
    t, h, v, m, g, T, D, mdot,
    vertical_rocket_equations,
    tsiolkovsky_delta_v,
)


## 1. Equations of motion

We request the symbolic equations of motion from `vertical_rocket_equations()`.


In [ ]:
eqs = vertical_rocket_equations()
eqs["h_dot"], eqs["v_dot"], eqs["m_dot"]

For readability, we can pretty-print them.


In [ ]:
for name, eq in eqs.items():
    print(f"--- {name} ---")
    display(sp.Eq(eq.lhs, eq.rhs))


The model is:

- $\dfrac{dh}{dt} = v$;
- $\dfrac{dv}{dt} = \dfrac{T - D(t) - m g}{m}$;
- $\dfrac{dm}{dt} = -\dot{m}$.

Here $D(t)$ is a generic drag term (we keep it symbolic for now),
and $\dot{m} > 0$ is the mass flow rate (mass decreases over time).


## 2. Mass evolution: solving $dm/dt = -\dot{m}$

The mass equation is linear and can be solved symbolically by `dsolve`.


In [ ]:
m_func = sp.Function("m")(t)
eq_m = sp.Eq(sp.diff(m_func, t), -mdot)
solution_m = sp.dsolve(eq_m)
solution_m

The general solution has the form:

$$
m(t) = C_1 - \dot{m} \cdot t.
$$

If we impose an initial condition $m(0) = m_0$, then $C_1 = m_0$ and:

$$
m(t) = m_0 - \dot{m} \cdot t.
$$


In [ ]:
m0 = sp.symbols("m0", positive=True)
C1 = sp.symbols("C1")

# From the dsolve result, extract C1 and impose m(0) = m0
m_t = solution_m.rhs.subs({C1: m0})
sp.simplify(m_t)


So, with an initial mass $m_0$ the mass decreases **linearly** with time:

$$
m(t) = m_0 - \dot{m} t.
$$

This is the simplest constant-mass-flow rocket model and the one used
by the numeric integrator in this lab.


## 3. Tsiolkovsky Δv: symbolic and numeric

The ideal rocket equation in vacuum is:

$$
\Delta v = I_{sp} \, g_0 \, \ln \left( \frac{m_0}{m_f} \right).
$$

We use `tsiolkovsky_delta_v` to get a SymPy expression and then
    evaluate it numerically for a simple example.


In [ ]:
Isp_val = 300.0     # s
g0_val = 9.80665    # m/s^2
m0_val = 500.0      # kg (wet mass)
mf_val = 200.0      # kg (dry mass)

dv_expr = tsiolkovsky_delta_v(Isp_val, g0_val, m0_val, mf_val)
dv_expr, float(sp.N(dv_expr))


The result can be compared to the direct numeric expression
$I_{sp} g_0 \ln(m_0 / m_f)$ and used as a reference when we later
integrate the full 1D equations of motion numerically.
